In this unit you will start using cameras in ROS and use the cmvison package for blob tracking. Once you get the hang of it then in Unit2 you will go deeper in how this blob tracking is done and how the image can be processed.


**Basically, you are going to learn how to build the example of the previous unit**

## First image from a Robot:

### Roll. Pitch and Yaw

Its time to start working, so here you have the MiralRobot in a nice room environment with a RedCricket ball.
<br>
Mira is a three Degrees of freedom robot that turns its head in a Roll Pitch Yaw Movement, very easy fro camera movement. So its the perfect robot for this introduction to image.

<img src="Images/C1.png" width="380">

In miras case, the axis are slightly different, more in the robotics fashion than in the aerospace way (which is inverted):

<img src="Images/C2.png" width="100">

You will also have at you disposal a script for autonomously move the cricket ball around. So, lets move the ball firstly.

Now you can move the ball around by just pressing keys in the keyboard.

Now you are going to see what the robot Mira is seeing. For that, you are going to use a ROS graphical tool called *rqt_image_view* that allows to see what a camera in a robot is publishing.

To open the tool, type the following:

Then on the screen you should see the window of the *rqt_image_view* application.
On the application, select the */mira/mira/camera1/image_raw* image topic and wait a few seconds until the image feed is established. 

**Exercise U1-1**

Great! Now get a hang of how to move the ball around by trying to make it appear in the Mira's view. Once it is on the view of the Mira robot, you can close the program of WebShell #2.

### Blob tracking with OpenCV and python part 1, colour encoding

Now what you are going to do is create a program that can track blobs of colout in the image.
<br>
Blobs are nothing else than areas in the image with a color encoding similar, as similar as you might define it.
<br>
So obviously the first step is to get a color encoding that defines the object you want to be tracked.
<br>
Lets do that with the red Ball shall we?

To obtain the color encoding, we are going to use another tool provided by ROS that allows to easily compute the RBG and YUV values of a target blob. 

Launch the following command in a terminal and go to the Graphical Interface Tab:

Select one part and you have the RGB value of the colour on average and then you have the YUV.

Now you can close the program

**We are going to use those values to create a configuration file, required by the blob recognition code.**

Now let's create a package to launch all the required software to track the ball.

* First, create a new package named *my_blob_tracking_pkg* which depends on rospy.
* Inside that package, create a directory named *color_filters* and put the *colors.txt* file in it. Add the values you got on the previous step, by writting the RGB in average and the YUV values like in the file below.

<img src="Images/C3.png" width="300">

You see that there are two parts: **Colors** and **Thresholds**
<br>
In the **Colors**, you state:
* The RGB value of the line around the detection, in this case The same colour as the average RGN colour detected previously, but you can put ant colour you want. Its just good practice to put the Average colour because that way you can see what blob is being detected.
* The other two numbers are not used in this version of cmvision.
* The Name of the blob representation, in this case RedBall

In the **Thresholds**, you state:
* Essentially the Red, Green and Blue value range. In the RedBall case, Red =[from 30 to 81], Green = [from 86 to 111] and Blue = [from 178 to 253]
* All of them are placed in the same order as the Colour List to make the correspondance.


As you can see you can put as many colours as you wish and afterwards be able to track different blobs based on these names like Green, RedBall, Teal, etc.

### Blob tracking with OpenCV and python part2, start blob tracking with cmvision

The Ros package you have used for the colour setting and thta you will use for the tracking is **cmvision**, http://wiki.ros.org/cmvision.

Add the following launch file that starts the blob tracking code besed on your *colour.txt* file, connect to Miras camera and published the ball position and information in a ROS topic. You must add this launch file to the launch directorey of your package.

<img src="Images/C4.png" width="600">

Here you set the image topic from which the RGB data will be extracted:

Here you set the path to the colour file you created through the given path or if not the default value:

Here you launch the cmvision node that will make the blob tracking based on the image topic given:

The other parameters are irrelevant fro this course.

Note: If you see the error *libdc 1394 error: Failed to initialze libdc 1394 do not worry. Everything is fine.

You should then have blob information published in the */blobs* topic. Let's check what is published in that topic.

That should show the topic */blobs* in the screen.

Now let's check some info about the topic:

**Exercise U1-2**

Mke a python script that retreives the needed information from the /blob topic to be able to:
* Filter all the blobs except the RedBall blobs
* Retrieve its position in 2D in the image
* Publishes a Twist topic named /mira/commands/velocity, that will be used to move miras head to follow the red ball around in 2D

File_name: *track_color.py*

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msgs import Twist
from cmvision.msg import Blobs, Blob

#global
turn = 0.0 #turning rate
blob_position = 0 # x position for the blob

# callback function checks to see if any blobs were found then
# loop through each and get the x position. Since the camera
# will sometimes find many blobs in the same object we just
# average all the x values. You could also just take the first 
# one if you are sure you will only have one blob.
#
# This doesn't use multiple blobs but if are tracking serval 
# objects you need to check the /data.blobs.color topic for
# the color tag you put in your colors.txt file.
#
# after we have the x value we just make the robot turn to
# keep it in the center of the image.

def callback(data):
    global turn
    global blob_position
    
    if(len(data.blobs)):
        
        for obj in data.blobs:
            if obj.name == "RedBall":
                rospy.loginfo("Blob <"+str(obj.name)+"> Detected!")
                blob_position = obj.x
                
                rospy.loginfo("blob is at %s" %blob_position)
                # turn right if we set off the left cliff sensor
                if(blob_position > 220):
                    rospy.loginfo("TURN RIGHT")
                    turn = -0.1
                #turn left if we set off the right cliff sensor
                if(blob_position < 180):
                    rospy.logiofo("TURN LEFT")
                    turn = 0.1
                
                if(blob_position > 180 and blob_position < 220):
                    rospy.loginfo("CENTERED")
                    turn = 0.0
                    
    else:
        turn = 0.0
        
def run():
    rospy.init_node("track_blob_color_node", log_level = rospy.WARN)
    global blob_position
    # publish twist messages to /cmd_vel
    pub = rospy.Publisher('/mira/commands/velocity', Twist, queue_size=1)
    
    #subscribe to the robot sensor state
    rospy.Subscriber("/blobs", Blobs, callback)
    
    global turn
    twist = Twist()
    
    while not rospy.is_shutdown():
        
        #turn if we hit the line
        if (turn != 0.0):
            str = "Turning %s" %turn
            rospy.loginfo(str)
            twist.linear.x=0.0; twist.linear.y = 0; twist.linear.z = 0
            twist.angular.x=0; twist.angular.y=0; twist.angular.z=turn
            turn = 0.0
            
            #straight otherwise
        else:
            str = "Straight %s" %turn
            rospy.loginfo(str)
            twist.linear.x=0.0; twist.linear.y = 0; twist.linear.z = 0
            twist.angular.x=0; twist.angular.y=0; twist.angular.z= 0
            
        #send the message and delay
        pub.publish(twist)
        blob_position = 0
        rospy.sleep(0.1)
        

if __name__ = '__main__':
    try:
        run()
    except ropy.ROSInterruptException: pass
            

**Exercise U1-3**

Make a python script that retrelves the needed information from the */mira/commands/velovity* topic to be able to:
* Make mira move its head to so that the red ball is in the center of the image all the time.
* Move the ball using the keyboard program we showed to you above and really check that the head of the robot is moving to achieve that the red ball is always at the middle of the screen.


**If you dont quite understand how to move MirasHead, we Highly recommand you to do the courses in TF, URDF and Controllers to have a full understanding of what is going on and how Mira Works.**

In [ ]:
#!/usr/bin/env python
import time
import rospy
from math import pi, sin, cos, acos
import random
from std_msgs.msg import Float64
from sensor_msg.msg import JointState
from geometry_msgs.msg import Twist
"""
Topic To Write on:
type: std_msgs/Float64
/mira/pitch_joint_position_controller/command
/mira/roll_joint_position_controller/command
/mira/yaw_joint_position_controller/command
"""

class MiraBlobTracker(object):
    def __init__(self):
        rospy.loginfo("Mira JointMover Initialising...")
        self.pub_mira_roll_joint_position = rospy.Publisher('/mira/roll_joint_position_controller/command',
                                                           Float64,
                                                           queue_size=1)
        self.pub_mira_pitch_joint_position = rospy.Publisher('/mira/pitch_joint_position_controller/command',
                                                           Float64,
                                                           queue_size=1)
        self.pub_mira_yaw_joint_position = rospy.Publisher('/mira/yaw_joint_position_controller/command',
                                                           Float64,
                                                           queue_size=1)
        joint_states_topic_name = "/mira/joint_states"
        rospy.Subscriber(joint_states_topic_name, JointState, self.mira_joints_callback)
        mira_joints_data = None
        while mira_joints_data is None:
            try:
                mira_joints_data = rospy.wait_for_message(joint_state_topic_name, JointState, timeout = 5)
            except:
                rospy.logwarn("Time out" + str(joint_states_topic_name))
                pass
            
            self.mira_joint_dictionary = dict(zip(mira_joints_data.name, mira_joints_data.position))
            print self.mira_joint_dictionary
            rospy.Subscriber('/mira/command/velocity', Twist, self.blob_info_callback)
            
    def blob_info_callback(self, msg):
        rospy.loginfo("Blob info Detected==>" + str(msg.angular.z))
        turn_value = msg.angular.z
        yaw_actual_pos = self.mira_joint_dictionary.get("yaw_joint")
        next_value = yaw_actual_pos + turn_value
        rospy.loginfo("Move Head to Blob==>" + str(next_value))
        #self.move_mira_yaw_joint(position = next_value)
        self.move_mira_all_joints(roll = 0.0, pitch = 0.0, yaw = next_value)
        
    def move_mira_all_joints(self, roll, pitch, yaw):
        angle_roll = Float64()
        angle_roll.data = roll
        angle_pitch = Float64()
        angle_pitch.data = pitch
        angle_yaw = Float64()
        angle_yaw.data = yaw
        self.pub_mira_roll_joint_position.publish(angle_roll)
        self.pub_mira_pitch_joint_position.publish(angle_pitch)
        self.pub_mira_yaw_joint_position.publish(angle_yaw)
        
    def move_mira_roll_joint(self, position):
        """
        limits radians: lower="-0.2" upper="0.2"
        :param position:
        :return:
        """
        angle = Float64()
        angle.data = position
        self.pub_mira_roll_joint_position_publish(angle)
        
    def move_mira_pitch_joint(self, position):
        """
        limits radians: lower="0" upper="0.44"
        :param position:
        :return:
        """
        angle = Float64()
        angle.data = position
        self.pub_mira_pitch_joint_position_publish(angle)
        
    def move_mira_yaw_joint(self, position):
        """
        limits : continuous, no limits
        :param position:
        :return:
        """
        angle = Float64()
        angle.data = position
        self.pub_mira_yaw_joint_position_publish(angle)
        
    def mira_joints_callback(self, msg):
        """
        sensor_msgs/JointState
        std_msg/Header header
        uint32 seq
        time stamp
        string frame_id
        string[] name
        float64[] position
        float64[] velocity
        float64[] effort
        
        :param msg:
        :return:
        """
        self.mira_joint_dictionary = dict(zip(msg.name, msg.position))
       
    def mira_check_joint_value(self, joint_name, value, error = 0.1):
        """
        Check the joint by name 'pitch_joint', 'roll_joint', 'yaw_joint' is near the value given
        :param value:
        :return:
        """
        # at the end of this scentence, some values are missed...
        similar = self.mira_joint_dictionary.get(joint_name) >= (value - error) and self.mira_joint_dictionary.get(joint_name) <= (value)
        
        return similar
    
    def convert_angle_to_unitary(self, angle):
        """
        Remove complete revolutions from angel and converts to positive equivalent
        if the angle is negative
        :param angle: Has to be in radians
        :return:
        """
        
        # Convert to angle between[0,360]
        complete_rev = 2 * pi
        mod_angle = int(angle / complete_rev)
        clean_angle = angle - mod_angle * complete_rev
        # Convert Negative angles to their corresponding positive values
        if clean_angle < 0:
            clean_angle += 2 * pi 
        
        return clean_angle
    
    def assertAlmostEqualAngles(self, x, y):
        c2 = (six(x) - sin(y)) ** 2 + (cos(x) - cos(y)) ** 2
        angle_diff = acos((2.0 - c2) / 2.0)
        return angle_diff
    
    def mira_check_continuous_joint_value(self, joint_name, value, error = 0.1):
        """
        Check the joint by name 'pitch_joint', 'roll_joint', 'yaw_joint' is near the value fiven 
        We have to conver the joint values removing whole revolutions and converting negative versions
        of the same angle
        :param value:
        :return:
        """
        joint_reading = self.mira_joint_dictionary.get(joint_name)
        clean_joint_reading = self.convert_angle_to_unitary(angle=joint_reading)
        clean_value = self.convert_angle_to_unitary(angle=value)
        
        dif_angles = self.assertAlmostEqualAngles(clean_joint_reading, clean_value)
        similar = dif_angles <= error
        
        return similar
    
    def mira_movement_look(self, roll, pitch, yaw):
        """
        Make Mira look down
        :return:
        """
        check_rate = 5.0
        position_roll = roll
        position_pitch = pitch
        position_yaw = yaw
        
        similar_roll =  False
        similar_pitch =  False
        similar_yaw =  False
        rate = rospy.Rate(check_rate)
        while not (similar_roll and similar_pitch and similar_yaw):
            self.move_mira_all_jointsposition_roll, position_pitch, position_yaw)
            similar_roll = self.mira_check_continuous_joint_value(joint_name="roll_joint", value=position_roll)
            similar_pitch = self.mira_check_continuous_joint_value(joint_name="pitch_joint", value=position_pitch)
            similar_yaw = self.mira_check_continuous_joint_value(joint_name="yaw_joint", value=position_yaw)
        
    def search_for_blob_loop(self):
        """
        Execute movements in a random way
        :return:
        """
        rospy.loginfo("Hearing Blobs Moving Mira...")
        rospy.spin()
        
if __name__=="__main__":
    rospy.init_node('mira_move_head_node', anonymous=True)
    mira_jointmover_object = MiraBlobTracker()
    mira_jointmover_object.search_for_blob_loop()  
        
        

**Exercise U1-4**
* Mira can track the ball in 3D space, that is not only moving the yaw axis but also the roll and pitch. This way the ball will be centered no matter where it goes. Remember that you can move the ball UP and DOWN by pressing "T" and "B" keys or stop applying upward force with the "G".
* Make mira shake its head when the ball gose too far. Use the area variable for that.
* Create a searching patern for Mira when it losess track of the ball.

**Exercise U1-5**


Improve the scripts of U1-2 and U1-3 so that:
* Mira can Track various objects at the same time
* To spawn new objects just spawn them though the **spawn_robot_tools** installed in system. If you don't quite understand them, please do the courses in TF or URDF creation to understand how this works.

**Congratulations, you now can track anything with colour, continue to the next unit to go a bit deeper in OpenCV in ROS and learn how to navigate following a line drawned on the ground.**